# meassurement of the accuracy 

accuracy is 
mean{ p in dotplot @ i,j  |  i,j in real_structure }

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')
import matplotlib.pyplot as plt

# Load data

In [ ]:
import ShaKer.rna_tools.rna_io as rio

def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

data = getdata()

for k,v in data.items():print k,"\t",  len(v[1])

# train predict

In [ ]:
import ShaKer.simushape as sim
from ShaKer.rna_tools.rnaplfold import rnaplfold

data = getdata()
model  = sim.make_model(data,['23sRNA','R009'],sim.make_forestregressor())
data.pop("23sRNA")
data.pop("R009")
keys = data.keys()

def get_str(seq):
    for _, data_seq, db in data.values():
        if seq == data_seq:
            return [db]
    print "SHIT"

# acc_calculator needs seq struct rea
predictions = { k: (data[k][1],data[k][2],sim.predict(model,data[k][1], seq_to_db_function=get_str)) for k in keys  }

shaker_plain = { k: (data[k][1],data[k][2],sim.predict(model,data[k][1])) for k in keys  }


# get accuracy

In [ ]:
from ShaKer.rna_tools.sukosd import sukosd
from ShaKer.rna_tools.rna_accuracy import get_structure_accuracy
import ShaKer.rna_tools.sukosd as suk

acc_shaker = {k: get_structure_accuracy (*predictions[k])  for  k in keys} 
acc_shaker_plain = {k: get_structure_accuracy(*shaker_plain[k]) for  k in keys} 
predictions_suko = { k: (data[k][1],data[k][2],sukosd(data[k][2])) for k in keys }

acc_suko = {k: get_structure_accuracy(*predictions_suko[k])  for  k in keys} 
acc_real = {k: get_structure_accuracy(data[k][1],data[k][2],data[k][0])  for  k in keys}
acc_noshape = {k: get_structure_accuracy(data[k][1],data[k][2],None)  for  k in keys}

predictions_suko_predicted = { k: (data[k][1],data[k][2],suk.predict_Suko(data[k][1])) for k in keys }
acc_suko_predicted = {k: get_structure_accuracy(*predictions_suko_predicted[k])  for  k in keys} 


acc_thoretical_best = {k: get_structure_accuracy(data[k][1],data[k][2],[1.0 if e =='.' else 0.0 for e in data[k][2]]) for k in keys} 



In [ ]:
maxmultistruct = { k: (data[k][1],data[k][2],suk.predict_tmp(data[k][1])) for k in keys }
maxmultistructt = {k: get_structure_accuracy(*maxmultistruct[k])  for  k in keys} 
#pandata.to_csv(r'pandas.txt', header=None, index=index, sep=' ', mode='a')

# Visualize 

In [ ]:
import pandas
import numpy as np
index= ['Sukosd',"ShaKer+Structure", "known_strucure_max","ShaKer","Thermodynamic",'Experimental',"Sukosd multistructure", "Max Multistructure"]


data = map(lambda x: [x[k] for k in keys] ,[acc_suko, acc_shaker,acc_thoretical_best,acc_shaker_plain, acc_noshape, acc_real,acc_suko_predicted,maxmultistructt])

#import notebook_helper as nbh
#nbh.boxplotDraw(data,index,"i am the fig title","","FUCKOFF")

import seaborn as sns
plt.xticks(rotation=70)
pandata = pandas.DataFrame(data, index=index).T
g=sns.boxplot(data=pandata)
plt.title("Accuracy")
g.set(ylim=(0, 1))
plt.grid()



In [ ]:

map(lambda x:x.append(np.mean(x)),data)
df = pandas.DataFrame(data, columns=keys+['mean'], index=index).T

print df.to_latex()

In [ ]:
from tabulate import tabulate 
print tabulate(df, headers="keys", tablefmt="orgtbl")

---